# **Hercoderen gemeenten 2022**

In 2022 zijn er een aantal gemeenten samengevoegd tot nieuwe gemeenten. Dit werkt door op een aantal plaatsen in de database:

2. In de data voor de Rioolwaterzuiveringsinstallaties (RWZI's). Waarschijnlijk wordt dit gewijzigd aangeleverd door RIVM, maar voor de zekerheid is er code ontwikkeld om dit op te vangen mocht de data in de oude indeling worden aangeleverd.



# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/utils/schemas.ipynb"
    ]
}
```

# **TABLES**

---


### STATICS

Om de gemeentelijke indeling te kunnen aanpassen naar de gewenste indeling wordt er een mapping tabel gemaakt die de wijzigingen bevat. Met behulp van deze mapping-tabel wordt de data die van VWSTAGE naar VWSSTATIC wordt gekopieerd, direct in de goede gemeentelijke indeling gezet. Dit gebeurt in de Stored Procedure **SP\_CBS\_POPULATION\_BASE\_STATIC**.

N.B. de kolom "Active" in de tabel Recode\_Municipality staat default op 0, wat betekent dat deze grenswijzigingen _NIET_ worden uitgevoerd. Om ze actief te maken moet met een update commando de waarde in de kolom "Active" op 1 worden gezet: 

Update VWSStatic.Recode\_Municipality SET Active = 1

FROM VWSStatic.Recode\_Municipality 

WHERE Date\_Last\_Inserted = (SELECT Max(Date\_Last\_Inserted) )

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

-- 1) CREATE STATIC TABLE(S).....
DROP TABLE IF EXISTS [VWSSTATIC].[RECLASSIFIED_MUNICIPALITY_MAPPING];
GO

-- THIS TABLE CONTAINS THE REQUIRED MAPPING REGARDING THE RECLASSIFICATIONS OF MULTIPLE MUNCIPALITIES.
CREATE TABLE [VWSSTATIC].[RECLASSIFIED_MUNICIPALITY_MAPPING] (
	[ID] [INT] PRIMARY KEY IDENTITY (1,1)
   ,DATE_LAST_INSERTED [DATETIME] NOT NULL DEFAULT GETDATE()
   ,[GM_CODE_ORIGINAL] [NVARCHAR](10)
   ,[GM_NAME_ORIGINAL] [NVARCHAR](255)
   ,[GM_CODE_NEW] [NVARCHAR](10)
   ,[GM_NAME_NEW] [NVARCHAR](255)
   ,[VR_CODE] [NVARCHAR](10)
   ,[VR_NAME] [NVARCHAR](255)
   ,[PROVINCE_CODE] [NVARCHAR](10)
   ,[PROVINCE_NAME] [NVARCHAR](255)
   ,[GGD_CODE] [NVARCHAR](10)
   ,[GGD_NAME] [NVARCHAR](255)
   ,[SHARES] [DECIMAL](16,6) NOT NULL DEFAULT (1.0)
   ,[ACTIVE] [INT] NOT NULL DEFAULT(0)
) ON [PRIMARY];
GO

INSERT INTO [VWSSTATIC].[RECLASSIFIED_MUNICIPALITY_MAPPING] (
   [GM_CODE_ORIGINAL],
   [GM_NAME_ORIGINAL],
   [GM_CODE_NEW],
   [GM_NAME_NEW],
   [VR_CODE],
   [VR_NAME],
   [PROVINCE_CODE],
   [PROVINCE_NAME],
   [GGD_CODE],
   [GGD_NAME],
   [SHARES],
   [ACTIVE])
VALUES 
   ('GM1684','Cuijk','GM1982','Land van Cuijk','VR21','Brabant-Noord','PV30','Noord-Brabant','GG5406','GGD Hart voor Brabant',1,0)
   ,('GM0756','Boxmeer','GM1982','Land van Cuijk','VR21','Brabant-Noord','PV30','Noord-Brabant','GG5406','GGD Hart voor Brabant',1,0)
	,('GM0815','Mill en Sint Hubert','GM1982','Land van Cuijk','VR21','Brabant-Noord','PV30','Noord-Brabant','GG5406','GGD Hart voor Brabant',1,0)
	,('GM1702','Sint Anthonis','GM1982','Land van Cuijk','VR21','Brabant-Noord','PV30','Noord-Brabant','GG5406','GGD Hart voor Brabant',1,0)
	,('GM0786','Grave','GM1982','Land van Cuijk','VR21','Brabant-Noord','PV30','Noord-Brabant','GG5406','GGD Hart voor Brabant',1,0)
	,('GM1685','Landerd','GM1991','Maashorst','VR21','Brabant-Noord','PV30','Noord-Brabant','GG5406','GGD Hart voor Brabant',1,0)
	,('GM0856','Uden','GM1991','Maashorst','VR21','Brabant-Noord','PV30','Noord-Brabant','GG5406','GGD Hart voor Brabant',1,0)
	,('GM0398','Heerhugowaard','GM1980','Dijk en Waard','VR10','Noord-Holland-Noord','PV27','Noord-Holland','GG2707','GGD Hollands-Noorden',1,0)
	,('GM0416','Langedijk','GM1980','Dijk en Waard','VR10','Noord-Holland-Noord','PV27','Noord-Holland','GG2707','GGD Hollands-Noorden',1,0)
	,('GM0370','Beemster','GM0439','Purmerend','VR11','Zaanstreek-Waterland','PV27','Noord-Holland','GG7306','GGD Zaanstreek/Waterland',1,0);
GO

# **OTHER**

---


Hier wordt gebruik gemaakt van de al aanwezige stored procedure `SP_MERGE_MOVE_STATIC_CBS_RWZI` om wijzigingen aan te brengen in de mapping van RWZI's naar gemeenten. Hiervoor wordt er een aantal regels toegevoegd aan het csv-bestand `Aantal_inwoners_per_verzorgingsgebied_mutaties**.csv` in de blob-storage, `datafiles\STATIC` map.

Inhoud van deze csv wordt dan:

```csv 
rwzi_code;rwzi_naam;startdatum;einddatum;inwoners;regio_type;regio_code;regio_naam;aandeel;toelichting

9043;LIENDEN;01-01-2020;07-12-2020;6728;GM;GM1740;Neder-Betuwe;1.5%;definitief

9043;LIENDEN;07-12-2020;31-12-2020;0;GM;GM1740;Neder-Betuwe;0.0%;definitief

9043;LIENDEN;01-01-2020;07-12-2020;6728;GM;GM0214;Buren;98.5%;definitief

9043;LIENDEN;07-12-2020;31-12-2020;0;GM;GM0214;Buren;0.0%;definitief

9025;TIEL;01-01-2020;07-12-2020;56015;GM;GM0214;Buren;5.2%;definitief

9025;TIEL;07-12-2020;31-12-2020;62813;GM;GM0214;Buren;15.3%;definitief

9025;TIEL;01-01-2020;07-12-2020;56015;GM;GM0281;Tiel;75.3%;definitief

9025;TIEL;07-12-2020;31-12-2020;62813;GM;GM0281;Tiel;66.7%;definitief

9025;TIEL;01-01-2020;07-12-2020;56015;GM;GM1740;Neder-Betuwe;13.7%;definitief

9025;TIEL;07-12-2020;31-12-2020;62813;GM;GM1740;Neder-Betuwe;12.7%;definitief

9025;TIEL;01-01-2020;07-12-2020;56015;GM;GM1960;West Betuwe;5.9%;definitief

9025;TIEL;07-12-2020;31-12-2020;62813;GM;GM1960;West Betuwe;5.3%;definitief

9029;AALST;01-01-2020;05-10-2020;10495;GM;GM0297;Zaltbommel;97.8%;definitief

9029;AALST;05-10-2020;31-12-2020;0;GM;GM0297;Zaltbommel;0.0%;definitief

9029;AALST;01-01-2020;05-10-2020;10495;GM;GM0263;Maasdriel;2.2%;definitief

9029;AALST;05-10-2020;31-12-2020;0;GM;GM0263;Maasdriel;0.0%;definitief

9041;ZALTBOMMEL;01-01-2020;05-10-2020;41974;GM;GM0263;Maasdriel;55.8%;definitief

9041;ZALTBOMMEL;05-10-2020;31-12-2020;53462;GM;GM0263;Maasdriel;45.1%;definitief

9041;ZALTBOMMEL;01-01-2020;05-10-2020;41974;GM;GM0297;Zaltbommel;44.2%;definitief

9041;ZALTBOMMEL;05-10-2020;31-12-2020;53462;GM;GM0297;Zaltbommel;54.9%;definitief

12014;BEEMSTER;01-01-2020;31-12-9999;104382;GM;GM0370;Beemster;0.0%;definitief

12041;BEEMSTER;01-01-2020;31-12-9999;104382;GM;GM0439;Purmerend;9.7%;definitief

12013;GEESTMERAMBACHT;01-01-2020;31-12-9999;185792;GM;GM0398;Heerhugowaard;0%;definitief

12013;GEESTMERAMBACHT;01-01-2020;31-12-9999;185792;GM;GM0416;Langedijk;0%;definitief

12013;GEESTMERAMBACHT;01-01-2020;31-12-9999;185792;GM;GM1980;Dijk en Waard;46.7%;definitief

29002;OIJEN;01-01-2020;31-12-9999;160856;GM;GM0786;Grave;0%;definitief

29003;LAND VAN CUIJK (HAPS);01-01-2020;31-12-9999;87802;GM;GM0756;Boxmeer;0%;definitief

29003;LAND VAN CUIJK (HAPS);01-01-2020;31-12-9999;87802;GM;GM0786;Grave;0%;definitief

29003;LAND VAN CUIJK (HAPS);01-01-2020;31-12-9999;87802;GM;GM0815;Mill en Sint Hubert;0%;definitief

29003;LAND VAN CUIJK (HAPS);01-01-2020;31-12-9999;87802;GM;GM1684;Cuijk;0%;definitief

29003;LAND VAN CUIJK (HAPS);01-01-2020;31-12-9999;87802;GM;GM1702;Sint Anthonis;0%;definitief

29002;OIJEN;01-01-2020;31-12-9999;160856;GM;GM1982;Land van Cuijk;6.91%;definitief

29003;LAND VAN CUIJK (HAPS);01-01-2020;31-12-9999;87802;GM;GM1982;Land van Cuijk;90.07%;definitief

28005;DINTHER;01-01-2020;31-12-9999;123719;GM;GM0856;Uden;0%;definitief

29002;OIJEN;01-01-2020;31-12-9999;160856;GM;GM1685;Landerd;0%;definitief

29003;LAND VAN CUIJK (HAPS);01-01-2020;31-12-9999;87802;GM;GM0856;Uden;0%;definitief

29003;LAND VAN CUIJK (HAPS);01-01-2020;31-12-9999;87802;GM;GM1685;Landerd;0%;definitief

28005;DINTHER;01-01-2020;31-12-9999;123719;GM;GM1991;Maashorst;32.60%;definitief

29002;OIJEN;01-01-2020;31-12-9999;160856;GM;GM1991;Maashorst;5.64%;definitief

29003;LAND VAN CUIJK (HAPS);01-01-2020;31-12-9999;87802;GM;GM1991;Maashorst;9.91%;definitief
```